In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
import re
import contractions
import seaborn as sns

import spacy
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

In [3]:
dia_full_df = pd.read_csv('/Users/JoKerry/Desktop/CSML1030 - Machine Learning Capstone/mimic_clinical_outcome_prediction/Concat DataFrames (Train, Test, Validate)/dia_full_df.csv')
dia_full_df

,id,text,short_codes
0,159643,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This 60 ...,"424,428,276,997,427,414,412,V458"
1,147171,CHIEF COMPLAINT: Substernal Chest Pain\n\nPRES...,"410,428,997,427,414,458"
2,199961,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"720,403,707,805,847,E885,780,285,250"
3,136812,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This is ...,"491,486,280,276,427,426,438,729,244"
4,175700,CHIEF COMPLAINT: s/p rollover MVC with prolong...,"823,481,861,518,285,813,807,E816,883,799,V641,..."
...,...,...,...
48736,132588,CHIEF COMPLAINT: Right sided weakness and dysa...,"431,348,486,518,401,359,365,276"
48737,183398,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"516,486,599,427,428,780,453,518,276"
48738,127889,CHIEF COMPLAINT: hip pain\n\nPRESENT ILLNESS: ...,"996,428,998,285,458,414,327,401,250,272,338,04..."
48739,171528,CHIEF COMPLAINT: Chest pain\n\nPRESENT ILLNESS...,"414,281,411,401,272,427,V586,V103,365,715,564,..."


## ICD_9 Labels {-}

In [4]:
def get_top_ICD9(s):
    code = s.split(',')[0]
    return code

In [5]:
dia_full_df['top_icd'] = [get_top_ICD9(x) for x in dia_full_df['short_codes']]

In [6]:
dia_full_df

,id,text,short_codes,top_icd
0,159643,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This 60 ...,"424,428,276,997,427,414,412,V458",424
1,147171,CHIEF COMPLAINT: Substernal Chest Pain\n\nPRES...,"410,428,997,427,414,458",410
2,199961,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"720,403,707,805,847,E885,780,285,250",720
3,136812,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This is ...,"491,486,280,276,427,426,438,729,244",491
4,175700,CHIEF COMPLAINT: s/p rollover MVC with prolong...,"823,481,861,518,285,813,807,E816,883,799,V641,...",823
...,...,...,...,...
48736,132588,CHIEF COMPLAINT: Right sided weakness and dysa...,"431,348,486,518,401,359,365,276",431
48737,183398,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"516,486,599,427,428,780,453,518,276",516
48738,127889,CHIEF COMPLAINT: hip pain\n\nPRESENT ILLNESS: ...,"996,428,998,285,458,414,327,401,250,272,338,04...",996
48739,171528,CHIEF COMPLAINT: Chest pain\n\nPRESENT ILLNESS...,"414,281,411,401,272,427,V586,V103,365,715,564,...",414


In [9]:
dia_full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48741 entries, 0 to 48740
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           48741 non-null  int64 
 1   text         48741 non-null  object
 2   short_codes  48741 non-null  object
 3   top_icd      48741 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


## Filtering ICD_9 Labels {-}

In [29]:
dia_df = dia_full_df.loc[dia_full_df['top_icd'].isin(['414', '038', '410', '424'])]
dia_df

,id,text,short_codes,top_icd
0,159643,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This 60 ...,"424,428,276,997,427,414,412,V458",424
1,147171,CHIEF COMPLAINT: Substernal Chest Pain\n\nPRES...,"410,428,997,427,414,458",410
6,190531,CHIEF COMPLAINT: chest pain\n\nPRESENT ILLNESS...,"414,428,584,486,411,585,276,250,403,440,V450,2...",414
11,140536,CHIEF COMPLAINT: Presented with respiratory di...,"410,486,276,425,414,427,695,401,416,496,356,43...",410
20,172808,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"410,998,997,427,414,362,V158",410
...,...,...,...,...
48722,100746,CHIEF COMPLAINT: Hematemesis and fevers\n\nPRE...,"038,507,599,999,578,438,995,451,E879,707,496,4...",038
48725,137725,"CHIEF COMPLAINT: Altered mental status, fever,...","038,599,584,591,276,293,518,250,294,287,280,40...",038
48728,159602,CHIEF COMPLAINT: Respiratory distress\n\nPRESE...,"038,518,486,584,427,008,496,424,995,519,V440,1...",038
48730,172011,CHIEF COMPLAINT: Chest pain and shortness of b...,"414,349,518,285,438,997,427,250,412,461,V498,2...",414


In [31]:
dia_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11537 entries, 0 to 48739
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           11537 non-null  int64 
 1   text         11537 non-null  object
 2   short_codes  11537 non-null  object
 3   top_icd      11537 non-null  object
dtypes: int64(1), object(3)
memory usage: 450.7+ KB


In [33]:
dia_df.top_icd.value_counts()

414    3502
038    3184
410    3175
424    1676
Name: top_icd, dtype: int64

In [34]:
dia_df

,id,text,short_codes,top_icd
0,159643,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This 60 ...,"424,428,276,997,427,414,412,V458",424
1,147171,CHIEF COMPLAINT: Substernal Chest Pain\n\nPRES...,"410,428,997,427,414,458",410
6,190531,CHIEF COMPLAINT: chest pain\n\nPRESENT ILLNESS...,"414,428,584,486,411,585,276,250,403,440,V450,2...",414
11,140536,CHIEF COMPLAINT: Presented with respiratory di...,"410,486,276,425,414,427,695,401,416,496,356,43...",410
20,172808,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"410,998,997,427,414,362,V158",410
...,...,...,...,...
48722,100746,CHIEF COMPLAINT: Hematemesis and fevers\n\nPRE...,"038,507,599,999,578,438,995,451,E879,707,496,4...",038
48725,137725,"CHIEF COMPLAINT: Altered mental status, fever,...","038,599,584,591,276,293,518,250,294,287,280,40...",038
48728,159602,CHIEF COMPLAINT: Respiratory distress\n\nPRESE...,"038,518,486,584,427,008,496,424,995,519,V440,1...",038
48730,172011,CHIEF COMPLAINT: Chest pain and shortness of b...,"414,349,518,285,438,997,427,250,412,461,V498,2...",414


In [35]:
dia_df.to_csv('dia_df_prepared.csv', index=False)